In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/daniloruberti/Desktop/train.csv')

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Handling missing values
for column in data.columns:
    if data[column].dtype == 'float64' or data[column].dtype == 'int64':
        data[column].fillna(data[column].mean(), inplace=True)
    else:
        data[column].fillna(data[column].mode()[0], inplace=True)
# Encoding categorical variables
label_encoder = LabelEncoder()
categorical_columns = ['Geography', 'Gender']
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])
# Normalizing numerical variables
numerical_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

/Users/daniloruberti/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/daniloruberti/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/daniloruberti/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/daniloruberti/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

In [4]:
data = data.drop(['Surname', 'CustomerId', 'id'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.144135,0,1,-0.578074,-0.719973,-0.883163,0.814298,1.0,0.0,181449.97,0
1,-0.367706,0,1,-0.578074,-1.432694,-0.883163,0.814298,1.0,1.0,49503.50,0
2,0.268974,0,1,0.211354,1.774548,-0.883163,0.814298,1.0,0.0,184866.69,0
3,-0.941966,0,1,-0.465299,-1.076334,1.486918,-1.013348,1.0,1.0,84560.88,0
4,0.743362,2,1,-0.578074,-0.007253,-0.883163,0.814298,1.0,1.0,15068.83,0


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [6]:
# Split data into features and target
X = data.drop('Exited', axis=1).values  
y = data['Exited'].values

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.float)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
class ChurnPredictor(nn.Module):
    def __init__(self):
        super(ChurnPredictor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [8]:
# Initialize the model, loss function, and optimizer
model = ChurnPredictor()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20  # Adjust as needed
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.2927188575267792
Epoch 2/20, Loss: 0.350214421749115
Epoch 3/20, Loss: 0.37348470091819763
Epoch 4/20, Loss: 0.3475818336009979
Epoch 5/20, Loss: 0.4089655578136444
Epoch 6/20, Loss: 0.27329763770103455
Epoch 7/20, Loss: 0.4155777096748352
Epoch 8/20, Loss: 0.21685531735420227
Epoch 9/20, Loss: 0.20365434885025024
Epoch 10/20, Loss: 0.33946698904037476
Epoch 11/20, Loss: 0.358146607875824
Epoch 12/20, Loss: 0.31698107719421387
Epoch 13/20, Loss: 0.33640217781066895
Epoch 14/20, Loss: 0.32808196544647217
Epoch 15/20, Loss: 0.4343706965446472
Epoch 16/20, Loss: 0.4142996072769165
Epoch 17/20, Loss: 0.4143621027469635
Epoch 18/20, Loss: 0.2778073847293854
Epoch 19/20, Loss: 0.25894829630851746
Epoch 20/20, Loss: 0.2576272785663605


In [9]:
# Ensure the model is in evaluation mode
model.eval()

# Store predictions and actual values
all_predictions = []
all_targets = []

with torch.no_grad():  
    for inputs, targets in test_loader:
        outputs = model(inputs)
        predictions = outputs.squeeze().round()  
        all_predictions.extend(predictions.numpy())
        all_targets.extend(targets.numpy())

# Convert lists to numpy arrays for metric calculations
all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(all_targets, all_predictions)
precision = precision_score(all_targets, all_predictions)
recall = recall_score(all_targets, all_predictions)
f1 = f1_score(all_targets, all_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


torch.save(model.state_dict(), 'model.pth')
!netron model.pth

Accuracy: 0.8638
Precision: 0.7580
Recall: 0.5198
F1 Score: 0.6167
Serving 'model.pth' at http://localhost:8081
